In [2]:
import pandas as pd

In [ ]:
# first load tables

domain_table = pd.read_csv('/Volumes/PGH-Backup/domains/IPS/test_cat/all_IPS_results.tsv', sep='\t', header=None)

display(domain_table, domain_table.shape)

In [16]:
# Group by the enzyme name and aggregate the domain column as a list
grouped_df = domain_table.groupby(0)[12].apply(list).reset_index()


In [ ]:
display(grouped_df)

In [18]:
grouped_df_clean = grouped_df

In [ ]:
grouped_df_clean[0] = grouped_df_clean[0].str.split('|').str[0]

display(grouped_df_clean)

In [22]:
grouped_df_clean.to_csv('/Volumes/PGH-Backup/domains/IPS/all_IPS_results_grouped.tsv', sep='\t', index=False, header=False)

**Begin Merging with cluster table**

In [9]:
## Temporary swap to a windows enviorment, paths changed accordingly
grouped_df_clean = pd.read_csv("E:\\domains\\IPS\\all_IPS_results_grouped.tsv", sep='\t', header=None)
display(grouped_df_clean)

,0,1
0,Amidase_UniRef100_A0A009ES59,['N-acetylmuramoyl-L-alanine amidase domain']
1,Amidase_UniRef100_A0A009FUX6,"['N-acetylmuramoyl-L-alanine amidase, catalyti..."
2,Amidase_UniRef100_A0A009H4S4,['N-acetylmuramoyl-L-alanine amidase domain']
3,Amidase_UniRef100_A0A009HT94,['N-acetylmuramoyl-L-alanine amidase domain']
4,Amidase_UniRef100_A0A009L0R9,['N-acetylmuramoyl-L-alanine amidase domain']
...,...,...
707784,UC118_WP_253005939.1,"['Endopeptidase, NLPC/P60 domain', 'LysM domai..."
707785,UC118_WP_255820014.1,"['LysM domain', 'LysM domain', 'LysM domain', ..."
707786,UC118_WP_263296879.1,"['Endopeptidase, NLPC/P60 domain', 'LysM domai..."
707787,UC118_WP_263297069.1,"['LysM domain', 'LysM domain', 'LysM domain', ..."


In [10]:
grouped_df_clean.rename(columns={0: 'Enzyme', 1: 'Domains'}, inplace=True)

In [11]:
grouped_df_clean['Uniref'] = grouped_df_clean['Enzyme'].str.split('_').str[2]

display(grouped_df_clean)

,Enzyme,Domains,Uniref
0,Amidase_UniRef100_A0A009ES59,['N-acetylmuramoyl-L-alanine amidase domain'],A0A009ES59
1,Amidase_UniRef100_A0A009FUX6,"['N-acetylmuramoyl-L-alanine amidase, catalyti...",A0A009FUX6
2,Amidase_UniRef100_A0A009H4S4,['N-acetylmuramoyl-L-alanine amidase domain'],A0A009H4S4
3,Amidase_UniRef100_A0A009HT94,['N-acetylmuramoyl-L-alanine amidase domain'],A0A009HT94
4,Amidase_UniRef100_A0A009L0R9,['N-acetylmuramoyl-L-alanine amidase domain'],A0A009L0R9
...,...,...,...
707784,UC118_WP_253005939.1,"['Endopeptidase, NLPC/P60 domain', 'LysM domai...",253005939.1
707785,UC118_WP_255820014.1,"['LysM domain', 'LysM domain', 'LysM domain', ...",255820014.1
707786,UC118_WP_263296879.1,"['Endopeptidase, NLPC/P60 domain', 'LysM domai...",263296879.1
707787,UC118_WP_263297069.1,"['LysM domain', 'LysM domain', 'LysM domain', ...",263297069.1


In [5]:
# read in cluster map
cluster_map = pd.read_csv("..\\..\\data\\clustering\\cluster_maps\\combined_clusters.tsv", sep='\t')

display(cluster_map)

C:\Users\odesa\AppData\Local\Temp\ipykernel_20284\3659657928.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,18,19,20,21,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  cluster_map = pd.read_csv("..\\..\\data\\clustering\\cluster_maps\\combined_clusters.tsv", sep='\t')


,dl_endopeptidase-unclustered,dl_endopeptidase-mmseqs_cluster,dl_endopeptidase-foldseek_cluster,ld_carboxypeptidase-unclustered,ld_carboxypeptidase-mmseqs_cluster,ld_carboxypeptidase-foldseek_cluster,ld_endopeptidase-unclustered,ld_endopeptidase-mmseqs_cluster,ld_endopeptidase-foldseek_cluster,glucosaminidase-unclustered,...,muramidase-foldseek_cluster,dd_carboxypeptidase-unclustered,dd_carboxypeptidase-mmseqs_cluster,dd_carboxypeptidase-foldseek_cluster,amidase-unclustered,amidase-mmseqs_cluster,amidase-foldseek_cluster,dd_endopeptidase-unclustered,dd_endopeptidase-mmseqs_cluster,dd_endopeptidase-foldseek_cluster
0,A0A7S8CCP0,A0A7S8CCP0,A0A7X2Z4H8,A0A964YMG5,A0A964YMG5,NaN,A0A0B8XNQ7,A0A0B8XNQ7,NaN,D7VR40,...,A0A1C6CUZ1,A0A420VF10,A0A6I2MG00,A0A1G7N0U6,A0A432IFZ8,A0A432IFZ8,A0A534TJ56,A0A0B8XNQ7,A0A0B8XNQ7,A0A0B0HVF0
1,A0A928LPM8,A0A413G4S5,A0A1C5Q6A7,A0A969WWE3,A0A7X8WLE8,A0A7X8WLE8,A0A0G1JBJ2,A0A0G1JBJ2,NaN,A0A2Z5VGL1,...,A0A1S8L485,A0A150MEC6,A0A6I2MG00,A0A1G7N0U6,A0A369Y4I6,A0A432IFZ8,A0A534TJ56,A0A0G1JBJ2,A0A0G1JBJ2,A4SWU3
2,A0A413G4S5,A0A413G4S5,A0A1C5Q6A7,A0A7X8WLE8,A0A7X8WLE8,A0A7X8WLE8,A0A0G1Q852,A0A0G1JBJ2,NaN,A0A3C1VCU0,...,A0A7V2DF21,A0A7X0LVZ4,A0A6I2MG00,A0A1G7N0U6,E3HA38,A0A432IFZ8,A0A534TJ56,A0A0G1Q852,A0A0G1JBJ2,A4SWU3
3,A0A926IJ38,A0A413G4S5,A0A1C5Q6A7,E0RLM3,E0RLM3,A0A2E9X5T2,A0A0J1ISN8,A0A0J1ISN8,NaN,A0A3D3DHY8,...,A0A7V2DF21,A0A061PFU4,A0A6I2MG00,A0A1G7N0U6,UPI0004050AE8,A0A369PUK8,A0A455SSC1,A0A0J1ISN8,A0A0J1ISN8,A0A0J1ISN8
4,A0A498CNA2,A0A413G4S5,A0A1C5Q6A7,S5LZK0,S5LZK0,A0A3E3E572,A0A0P9GGU0,A0A0P9GGU0,NaN,A0A424PX64,...,A0A534QK47,A0A841PZK5,A0A6I2MG00,A0A1G7N0U6,A0A924USN5,A0A369PUK8,A0A455SSC1,A0A0P9GGU0,A0A0P9GGU0,A0A840V6T9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# melt the cluster map

cluster_map_melted = pd.melt(cluster_map, var_name='column_name', value_name='uniref_id')

display(cluster_map_melted)

,column_name,uniref_id
0,dl_endopeptidase-unclustered,A0A7S8CCP0
1,dl_endopeptidase-unclustered,A0A928LPM8
2,dl_endopeptidase-unclustered,A0A413G4S5
3,dl_endopeptidase-unclustered,A0A926IJ38
4,dl_endopeptidase-unclustered,A0A498CNA2
...,...,...
9276790,dd_endopeptidase-foldseek_cluster,NaN
9276791,dd_endopeptidase-foldseek_cluster,NaN
9276792,dd_endopeptidase-foldseek_cluster,NaN
9276793,dd_endopeptidase-foldseek_cluster,NaN


In [7]:
# Extract the enzyme name and cluster type (unclustered, mmseqs_cluster, foldseek_cluster)
cluster_map_melted['enzyme'] = cluster_map_melted['column_name'].apply(lambda x: x.split('-')[0])  # Extract enzyme name
cluster_map_melted['cluster_type'] = cluster_map_melted['column_name'].apply(lambda x: x.split('-')[1])  # Extract cluster type

display(cluster_map_melted)

,column_name,uniref_id,enzyme,cluster_type
0,dl_endopeptidase-unclustered,A0A7S8CCP0,dl_endopeptidase,unclustered
1,dl_endopeptidase-unclustered,A0A928LPM8,dl_endopeptidase,unclustered
2,dl_endopeptidase-unclustered,A0A413G4S5,dl_endopeptidase,unclustered
3,dl_endopeptidase-unclustered,A0A926IJ38,dl_endopeptidase,unclustered
4,dl_endopeptidase-unclustered,A0A498CNA2,dl_endopeptidase,unclustered
...,...,...,...,...
9276790,dd_endopeptidase-foldseek_cluster,NaN,dd_endopeptidase,foldseek_cluster
9276791,dd_endopeptidase-foldseek_cluster,NaN,dd_endopeptidase,foldseek_cluster
9276792,dd_endopeptidase-foldseek_cluster,NaN,dd_endopeptidase,foldseek_cluster
9276793,dd_endopeptidase-foldseek_cluster,NaN,dd_endopeptidase,foldseek_cluster


In [8]:
df_pivoted = cluster_map_melted.pivot_table(index=['enzyme'], columns=['cluster_type'], values='uniref_id', aggfunc='first').reset_index()

display(df_pivoted)

cluster_type,enzyme,foldseek_cluster,mmseqs_cluster,unclustered
0,amidase,A0A534TJ56,A0A432IFZ8,A0A432IFZ8
1,dd_carboxypeptidase,A0A1G7N0U6,A0A6I2MG00,A0A420VF10
2,dd_endopeptidase,A0A0B0HVF0,A0A0B8XNQ7,A0A0B8XNQ7
3,diadenylate_cyclase,A0A2M7JD51,A0A132I377,A0A939XBN5
4,dl_endopeptidase,A0A7X2Z4H8,A0A7S8CCP0,A0A7S8CCP0
5,glucosaminidase,A0A318MMB0,A0A380CQR0,D7VR40
6,ld_carboxypeptidase,A0A7X8WLE8,A0A964YMG5,A0A964YMG5
7,ld_endopeptidase,A0A0Q0GFJ5,A0A0B8XNQ7,A0A0B8XNQ7
8,muramidase,A0A1C6CUZ1,A0A7U9KRU0,A0A7U9KRU0
